In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from numpy import asarray
from PIL import Image
%matplotlib inline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


In [ ]:
import os
import random
import warnings
warnings.filterwarnings('ignore')

In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
# Используемые директории
directory = "C:/Users/hev20/OneDrive/Рабочий стол/Мага 1сем/flowers/"
directory_test = "C:/Users/hev20/OneDrive/Рабочий стол/Мага 1сем/flowers_test/"



In [ ]:
# Тренировочные данные
train_data = tf.keras.preprocessing.image_dataset_from_directory(
             directory,
             subset='training',
             validation_split=0.2,
             image_size=IMAGE_SIZE,
             batch_size=BATCH_SIZE,
             seed=42)


In [ ]:
# Валидационные данные
valid_data = tf.keras.preprocessing.image_dataset_from_directory(
            directory,
            subset='validation',
            validation_split=0.2,
            image_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
             seed=42)


In [ ]:
class_names = train_data.class_names
class_names


In [ ]:
label_encode = LabelEncoder()
class_names_label_encode = label_encode.fit_transform(class_names)
class_names_label_encode


In [ ]:
def preprocess_image(image, label, image_shape=224):
    
    img = tf.image.resize(image, [image_shape, image_shape])
    img = img/225.
    
    return tf.cast(img, tf.float32), label


In [ ]:
# map the preprocess_image to train_data
train_data = train_data.map(map_func=preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
# shuffle the data
train_data = train_data.shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.AUTOTUNE)

# map the preprocess_image to valid_data
valid_data = valid_data.map(map_func=preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
# shuffle the data
valid_data = valid_data.shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Set random seed
tf.random.set_seed(42)

In [ ]:
# model 3
model_5 = Sequential([
    Conv2D(filters=32, kernel_size=4, padding='same', activation='relu',input_shape=(224,224,3)),
    MaxPool2D(2,2),
    Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'),
    MaxPool2D(2,2), 
    Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'),
    MaxPool2D(2,2),
    Conv2D(filters=128, kernel_size=4, padding='same', activation='relu'),
    MaxPool2D(2,2),
    # Dropout(0.5),
    Flatten(),
    # Dense(512, activation='relu'),
    Dense(len(class_names_label_encode), activation='softmax')
])

In [ ]:
# Compile
model_5.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
               optimizer='adam',
               metrics=['accuracy'])

# Fit
history_1 = model_5.fit(train_data,
                       epochs=10,
                       validation_data=valid_data)

In [ ]:
def plot_loss_curves(history):
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(len(history.history['loss']))

  # Plot loss
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()
  # Plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()

plot_loss_curves(history=history_1)



In [ ]:
im, tit = preprocess_image(image=img, label='rose')
print(im.shape)
im = im[np.newaxis, :, :, :]



In [ ]:
probability_model = tf.keras.Sequential([model_5, 
                                         tf.keras.layers.Softmax()])


In [ ]:
predictions = probability_model.predict(im)
print(model_5.predict(im))

class_names[np.argmax(predictions[0])]

img = plt.imread(directory_test + "dandelion2.jpg")
plt.imshow(img)
plt.title("prediction: " + str(class_names[np.argmax(predictions[0])]))
plt.axis("off")
plt.show()


In [ ]:
# Download Inception V3 model
base_model_inception = tf.keras.applications.inception_v3.InceptionV3(include_top=False)

# Freeze the layers
base_model_inception.trainable=False

# Inputs
inputs = tf.keras.layers.Input(shape=(224,224,3), name='input_layer')

# Scaling the values
x = tf.keras.layers.experimental.preprocessing.Rescaling(1/255.)(inputs)

# Pass inputs to our base_model
x = base_model_inception(inputs,training=False)

# GlobalAveragePooling2D
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# outputs
outputs = tf.keras.layers.Dense(len(class_names_label_encode), activation='softmax')(x)

# Build model
model_v3 = tf.keras.Model(inputs, outputs)


In [ ]:
# Compile the model
model_v3.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
               optimizer='adam',
               metrics=['accuracy'])

# Fit the model
history_v3 = model_v3.fit(train_data,
                       epochs=10,
                       validation_data=valid_data)

plot_loss_curves(history_v3)

im, tit = preprocess_image(image=img, label='rose')
print(im.shape)
im = im[np.newaxis, :, :, :]

probability_model = tf.keras.Sequential([model_v3, 
                                         tf.keras.layers.Softmax()])




In [ ]:
predictions = probability_model.predict(im)
print(model_v3.predict(im))

In [ ]:
class_names[np.argmax(predictions[0])]

img = plt.imread(directory_test + "sunflower1.jpg")
plt.imshow(img)
plt.title("prediction: " + str(class_names[np.argmax(predictions[0])]))
plt.axis("off")
plt.show()
